In [ ]:
!nvidia-smi

Mon Feb  7 09:56:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! gdown --id 1i2x7osfloYkXDHMRn0WuVs2P3m751cj1
! pip -q uninstall -y kaggle
! pip -q install --upgrade pip
! pip -q install kaggle --upgrade
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download jigsaw-toxic-severity-rating
# ! kaggle datasets download aishikai/js-roberta
! kaggle datasets download aishikai/js-cleaned-validation-data

Downloading...
From: https://drive.google.com/uc?id=1i2x7osfloYkXDHMRn0WuVs2P3m751cj1
To: /content/kaggle.json
100% 64.0/64.0 [00:00<00:00, 108kB/s]
     |████████████████████████████████| 2.1 MB 13.9 MB/s 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 KB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
 74% 5.00M/6.72M [00:00<00:00, 50.1MB/s]
100% 6.72M/6.72M [00:00<00:00, 63.4MB/s]
  0% 0.00/7.35M [00:00<?, ?B/s]
100% 7.35M/7.35M [00:00<00:00, 202MB/s]


In [ ]:
!mkdir Data
!unzip /content/jigsaw-toxic-severity-rating.zip -d Data
!rm /content/jigsaw-toxic-severity-rating.zip
!mkdir roberta
# !unzip /content/js-roberta.zip -d roberta
# !rm /content/js-roberta.zip
! unzip /content/js-cleaned-validation-data.zip -d JTSR
! rm /content/js-cleaned-validation-data.zip

Archive:  /content/jigsaw-toxic-severity-rating.zip
  inflating: Data/comments_to_score.csv  
  inflating: Data/sample_submission.csv  
  inflating: Data/validation_data.csv  
Archive:  /content/js-cleaned-validation-data.zip
  inflating: JTSR/Validation_data_clean.csv  
  inflating: JTSR/Validation_data_clean_no_duplicates.csv  
  inflating: JTSR/submission.csv     


In [ ]:
! pip -q install sentencepiece
! pip -q install transformers
! pip -q install wandb --upgrade
! pip -q install colorama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 KB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.2/895.2 KB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.9/180.9 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 KB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 KB 7.8 MB/s eta 0:00:00


In [ ]:
import os
import gc
import copy
import time
import random
import string
import wandb

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW, AutoConfig

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
Config = {"seed": 2021,
          "csv_path":"/content/JTSR/Validation_data_clean.csv",
          "epochs": 5,
          "model_name": "sentence-transformers/all-mpnet-base-v2",
          "size": 768,
          "train_batch_size": 64,
          "valid_batch_size": 64,
          "num_workers": 4,
          "max_length": 128,
          "lr": 1e-5,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,  
          "T_0":20,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          'wandb': True,
          'apex': True,
          'n_accum': 1,
          "debug":False
          }

Config["tokenizer"] = AutoTokenizer.from_pretrained(Config['model_name'])

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(Config['seed'])

In [ ]:
# Start W&B logging
if Config['wandb']:
    run = wandb.init(
        project='pytorch',
        config=Config,
        group='nlp',
        job_type='train',
        anonymous='must'
    )

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Some utility functions
def wandb_log(**kwargs):
    """
    Logs a key-value pair to W&B
    """
    for k, v in kwargs.items():
        wandb.log({k: v})

In [ ]:
class UnionFind():
    def __init__(self, n):
        self.n = n
        self.parents = [-1] * n

    def find(self, x):
        if self.parents[x] < 0:
            return x
        else:
            self.parents[x] = self.find(self.parents[x])
            return self.parents[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)
        if x == y:
            return
        if self.parents[x] > self.parents[y]:
            x, y = y, x
        self.parents[x] += self.parents[y]
        self.parents[y] = x


def get_group_unionfind(train: pd.DataFrame):
    unique_text = set(train['less_toxic']) | set(train['more_toxic'])
    text2num = {text: i for i, text in enumerate(unique_text)}
    num2text = {num: text for text, num in text2num.items()}
    train['num_less_toxic'] = train['less_toxic'].map(text2num)
    train['num_more_toxic'] = train['more_toxic'].map(text2num)

    uf = UnionFind(len(unique_text))
    for seq1, seq2 in train[['num_less_toxic', 'num_more_toxic']].to_numpy():
        uf.union(seq1, seq2)

    text2group = {num2text[i]: uf.find(i) for i in range(len(unique_text))}
    train['group'] = train['less_toxic'].map(text2group)
    train = train.drop(columns=['num_less_toxic', 'num_more_toxic'])
    return train

In [ ]:
def get_folds(df, n_splits=Config['n_fold']):
    df = get_group_unionfind(df)
    group_kfold = GroupKFold(n_splits = n_splits)
    for fold, (trn_idx, val_idx) in enumerate(group_kfold.split(df, df, df['group'])): 
        df.loc[val_idx , "kfold"] = fold

    df["fold"] = df["kfold"].astype(int)
    return df

In [ ]:
class JigsawDataset(Dataset):
    def __init__(self, df, tokenizer=Config["tokenizer"], max_length=Config["max_length"]):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.more_toxic = df['more_toxic'].values
        self.less_toxic = df['less_toxic'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        more_toxic = self.more_toxic[index]
        less_toxic = self.less_toxic[index]
        inputs_more_toxic = self.tokenizer.encode_plus(
                                more_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        inputs_less_toxic = self.tokenizer.encode_plus(
                                less_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        target = 1
        
        more_toxic_ids = inputs_more_toxic['input_ids']
        more_toxic_mask = inputs_more_toxic['attention_mask']
        
        less_toxic_ids = inputs_less_toxic['input_ids']
        less_toxic_mask = inputs_less_toxic['attention_mask']
        
        
        return {
            'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.long),
            'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.long),
            'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.long),
            'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
class JigsawModel(nn.Module):
    def __init__(self, model_name):
        super(JigsawModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(Config['size'],256)
        self.fc2 = nn.Linear(256,Config['num_classes'])

    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc2(self.fc1(out))
        return outputs

In [ ]:
# class AttentionHead(nn.Module):
#     def __init__(self, in_features, hidden_dim):
#         super().__init__()
#         self.in_features = in_features
#         self.middle_features = hidden_dim
#         self.W = nn.Linear(in_features, hidden_dim)
#         self.V = nn.Linear(hidden_dim, 1)
#         self.out_features = hidden_dim

#     def forward(self, features):
#         att = torch.tanh(self.W(features))
#         score = self.V(att)
#         attention_weights = torch.softmax(score, dim=1)
#         context_vector = attention_weights * features
#         context_vector = torch.sum(context_vector, dim=1)

#         return context_vector

# class JigsawModel(nn.Module):
#     def __init__(self, model_name, config_path=None, pretrained=False):
#         super().__init__()
#         self.config = AutoConfig.from_pretrained(model_name, output_hidden_states=True)
#         self.model = AutoModel.from_pretrained(model_name, config=self.config)
#         self.head = AttentionHead(self.config.hidden_size,self.config.hidden_size)
#         self.dropout = nn.Dropout(0.4)
#         self.linear = nn.Linear(self.config.hidden_size,1)

#     def forward(self, ids, mask):
#         x = self.model(input_ids=ids,attention_mask=mask)[0]
#         x = self.head(x)
#         x = self.dropout(x)
#         x = self.linear(x)
#         return x

In [ ]:
def criterion(outputs1, outputs2, targets):
    return nn.MarginRankingLoss(margin=Config['margin'])(outputs1, outputs2, targets)

In [ ]:
def accuracy(y_more_toxic, y_less_toxic): 
    a = np.sum(np.where(y_more_toxic>y_less_toxic, 1, 0))
    b = np.sum(np.where(y_more_toxic>y_less_toxic, 0, 1))
    return a / (a + b)

In [ ]:
class Trainer:
    def __init__(self, config, dataloaders, optimizer, model, loss_fns, scheduler, device=Config['device'], apex=Config['apex']):
        self.train_loader, self.valid_loader = dataloaders
        self.train_loss_fn, self.valid_loss_fn = loss_fns
        self.scheduler = scheduler
        self.optimizer = optimizer
        self.model = model
        self.device = device
        self.apex = apex
        self.Config = config
    
    def train_one_epoch(self):
        if self.apex:
            scaler = GradScaler()

        self.model.train()
        train_pbar = tqdm(enumerate(self.train_loader), total=len(self.train_loader))
        train_preds, train_labels = [], []
        dataset_size = 0
        running_loss = 0.0

        for step, data in train_pbar:        
            more_toxic_ids = data['more_toxic_ids'].to(self.device, dtype = torch.long)
            more_toxic_mask = data['more_toxic_mask'].to(self.device, dtype = torch.long)
            less_toxic_ids = data['less_toxic_ids'].to(self.device, dtype = torch.long)
            less_toxic_mask = data['less_toxic_mask'].to(self.device, dtype = torch.long)
            targets = data['target'].to(self.device, dtype=torch.long)
            
            batch_size = more_toxic_ids.size(0)

            if self.apex:
                with(autocast(enabled=True)):

                    more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
                    less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
                    
                    loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
                    loss = loss / self.Config["n_accum"]
                
                scaler.scale(loss).backward()

                if (step + 1) % self.Config["n_accum"] == 0:
                    scaler.step(self.optimizer)
                    scaler.update()
                    optimizer.zero_grad()

                    if self.scheduler:
                        self.scheduler.step()

                running_loss += (loss.item() * batch_size)
                dataset_size += batch_size
            
            else:
                more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
                less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
                
                loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
                loss = loss / self.Config["n_accum"]

                loss.backward()
                loss = loss / self.Config["n_accum"]
                if (step + 1)%self.Config["n_accum"] == 0:
                    optimizer.step()
                    optimizer.zero_grad()

                    if self.scheduler:
                        self.scheduler.step()

                running_loss += (loss.item() * batch_size)
                dataset_size += batch_size

            epoch_loss = running_loss / dataset_size                
            
            train_pbar.set_postfix(Train_Loss=epoch_loss,
                            LR=optimizer.param_groups[0]['lr'])   
        
        gc.collect()
        
        return epoch_loss

    @torch.no_grad()
    def valid_one_epoch(self):

        self.model.eval()
        valid_pbar = tqdm(enumerate(self.valid_loader), total=len(self.valid_loader))
        more_toxic_preds = []
        less_toxic_preds = []
        dataset_size = 0
        running_loss = 0.0

        for step, data in valid_pbar:
            more_toxic_ids = data['more_toxic_ids'].to(self.device, dtype = torch.long)
            more_toxic_mask = data['more_toxic_mask'].to(self.device, dtype = torch.long)
            less_toxic_ids = data['less_toxic_ids'].to(self.device, dtype = torch.long)
            less_toxic_mask = data['less_toxic_mask'].to(self.device, dtype = torch.long)
            targets = data['target'].to(self.device, dtype=torch.long)
            
            batch_size = more_toxic_ids.size(0)

            more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
            less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)

            loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size
            more_toxic_preds.append(more_toxic_outputs.sigmoid().view(-1).cpu().detach().numpy())
            less_toxic_preds.append(less_toxic_outputs.sigmoid().view(-1).cpu().detach().numpy())
            epoch_loss = running_loss / dataset_size
            
            
            valid_pbar.set_postfix(Valid_Loss=epoch_loss,
                            LR=optimizer.param_groups[0]['lr'])
        
            gc.collect()
        
        more_toxic_preds = np.concatenate(more_toxic_preds)
        less_toxic_preds = np.concatenate(less_toxic_preds)

        return more_toxic_preds, less_toxic_preds, dataset_size, epoch_loss
            

    def fit(self, fold: str, epochs: int = 10, output_dir: str = "/content/models/", custom_name: str = 'model.pth'):
        """
        Low-effort alternative for doing the complete training and validation process
        """
        best_loss = int(1e+7)
        for epx in range(epochs):
            print(f"{'='*20} Epoch: {epx+1} / {epochs} {'='*20}")

            train_loss = self.train_one_epoch()
            print(f"Training loss: {train_loss:.4f}")

            more_toxic_preds, less_toxic_preds, length, valid_loss = self.valid_one_epoch()
            print(f'Validation Score: {accuracy(more_toxic_preds, less_toxic_preds):.4f}')
            print(f"Validation loss: {valid_loss:.4f}")

            if valid_loss < best_loss:
                best_loss = valid_loss
                self.save_model(output_dir, custom_name)
                print(f"Saved model with val_loss: {best_loss:.4f}")
            

            # Log
            if Config['wandb']:
                wandb_log(
                    train_loss=train_loss,
                    val_loss=valid_loss
                )

    def save_model(self, path, name, verbose=False):
        """
        Saves the model at the provided destination
        """
        try:
            if not os.path.exists(path):
                os.makedirs(path)
        except:
            print("Errors encountered while making the output directory")

        torch.save(self.model.state_dict(), os.path.join(path, name))
        if verbose:
            print(f"Model Saved at: {os.path.join(path, name)}")

In [ ]:
if __name__ == '__main__':
    train_file = pd.read_csv(Config['csv_path'])
    train_file = get_folds(train_file)
    
    if Config['debug']:
        train_file=train_file[:200]
        
    for fold_ in range(Config["n_fold"]):
        print(f"{'='*40} Fold: {fold_+1} / {Config['n_fold']} {'='*40}")
        
        train_ = train_file.loc[train_file["kfold"]!=fold_]
        valid_ = train_file.loc[train_file["kfold"]==fold_]
        
        train_set = JigsawDataset(
            df = train_
        )
        valid_set = JigsawDataset(
            df = valid_
        )
        
        train_loader = DataLoader(
            train_set,
            batch_size = Config['train_batch_size'],
            shuffle = True,
            num_workers = Config['num_workers'],
            pin_memory = True
        )
        
        valid_loader = DataLoader(
            valid_set,
            batch_size = Config['valid_batch_size'],
            shuffle = False,
            num_workers = Config['num_workers'],
        )
        
        model = JigsawModel(model_name=Config['model_name'])
        model = model.to(torch.device(Config['device']))
        # model.load_state_dict(torch.load(f"/content/hateBERT/JS_pre_hateBERT/models/hateBERT_{fold_}_model.bin"))
        if Config['wandb']:
            wandb.watch(model)
            
        optimizer = torch.optim.AdamW(model.parameters(), lr=Config['lr'])
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, 
            T_0=Config['T_0'], 
            eta_min=Config['min_lr']
        )
        train_lfn, valid_lfn = criterion, criterion
        
        trainer = Trainer(
            config = Config,
            dataloaders=(train_loader, valid_loader),
            loss_fns=(train_lfn, valid_lfn),
            optimizer=optimizer,
            model = model,
            scheduler=scheduler,
            apex=True
        )
        
        trainer.fit(
            fold = fold_,
            epochs = Config['epochs'],
            custom_name = f"{Config['model_name'].split('/')[-1]}_{fold_}_model.bin"
        )

======================================== Fold: 1 / 5 ========================================


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

==================== Epoch: 1 / 5 ====================


100%|██████████| 311/311 [02:16<00:00,  2.28it/s, LR=4.8e-6, Train_Loss=0.319]


...

Validation Score: 0.7472
Validation loss: 0.3220
==================== Epoch: 5 / 5 ====================



100%|██████████| 311/311 [02:14<00:00,  2.31it/s, LR=2.32e-6, Train_Loss=0.0992]


Training loss: 0.0992


100%|██████████| 78/78 [00:36<00:00,  2.11it/s, LR=2.32e-6, Valid_Loss=0.353]


Validation Score: 0.7408
Validation loss: 0.3533


In [ ]:
!zip -r /content/drive/MyDrive/JS_mpnetv2-base_RL.zip models

  adding: models/ (stored 0%)
  adding: models/all-mpnet-base-v2_4_model.bin (deflated 8%)
  adding: models/all-mpnet-base-v2_0_model.bin (deflated 8%)
  adding: models/all-mpnet-base-v2_3_model.bin (deflated 8%)
  adding: models/all-mpnet-base-v2_2_model.bin (deflated 8%)
  adding: models/all-mpnet-base-v2_1_model.bin (deflated 8%)


In [ ]:
# !zip -r /content/drive/MyDrive/JS_Deverta-v3B_RL.zip models

In [ ]:
# !rm -r /content/models

In [ ]:
# MODEL_PATHS = [f"/content/roberta/JS_roberta-base_RL/models/roberta-base_{fold}_model.bin" for  fold in range(5)]

In [ ]:
MODEL_PATHS = [f"/content/models/all-mpnet-base-v2_{fold}_model.bin" for  fold in range(5)]

In [ ]:
val_df = pd.read_csv("/content/JTSR/Validation_data_clean_no_duplicates.csv")
val_df.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [ ]:
class JigsawDatasetTest(Dataset):
    def __init__(self, df, tokenizer, max_length, col):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df[col].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
                        text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding='max_length'
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']        
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)
        }

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = JigsawModel(Config["model_name"])
        model.to(Config['device'])
        model.load_state_dict(torch.load(path, map_location = device))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        PREDS.append(outputs.sigmoid().view(-1).cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    del model
    gc.collect()
    
    return PREDS

In [ ]:
test_dataset = JigsawDatasetTest(val_df, Config['tokenizer'], max_length=Config['max_length'], col = "more_toxic")
test_loader = DataLoader(test_dataset, batch_size=Config['valid_batch_size'],
                            num_workers=2, shuffle=False, pin_memory=True)
preds_more = inference(MODEL_PATHS , test_loader, Config['device'])

Getting predictions for model 1


100%|██████████| 388/388 [01:00<00:00,  6.42it/s]


Getting predictions for model 2


100%|██████████| 388/388 [01:01<00:00,  6.27it/s]


Getting predictions for model 3


100%|██████████| 388/388 [01:01<00:00,  6.35it/s]


Getting predictions for model 4


100%|██████████| 388/388 [01:01<00:00,  6.32it/s]


Getting predictions for model 5


100%|██████████| 388/388 [01:00<00:00,  6.38it/s]


In [ ]:
test_dataset = JigsawDatasetTest(val_df, Config['tokenizer'], max_length=Config['max_length'], col = "less_toxic")
test_loader = DataLoader(test_dataset, batch_size=Config['valid_batch_size'],
                            num_workers=2, shuffle=False, pin_memory=True)
preds_less = inference(MODEL_PATHS , test_loader, Config['device'])

Getting predictions for model 1


100%|██████████| 388/388 [01:01<00:00,  6.34it/s]


Getting predictions for model 2


100%|██████████| 388/388 [01:01<00:00,  6.35it/s]


Getting predictions for model 3


100%|██████████| 388/388 [01:01<00:00,  6.35it/s]


Getting predictions for model 4


100%|██████████| 388/388 [01:00<00:00,  6.37it/s]


Getting predictions for model 5


100%|██████████| 388/388 [01:01<00:00,  6.36it/s]


In [ ]:
predictions = preds_more > preds_less
val_df["truth"] = 1
val_truths = val_df.truth.values
print(f"Accuracy: {accuracy_score(val_truths, predictions)}")

Accuracy: 0.8052241212512092
